# pFBA & MOMA: Compare flux distribution WT vs ACSBG#31 mutant

Strategy: 
- pFBA: each BOF is constrainded to experimental growth value and photon uptake is minimized.

- MOMA: pFBA WT is used as reference.

### Directories

In [ ]:
GEM_folder = "..\\..\\..\\..\\"

In [ ]:
modelfile = "model\\iMgadit23.json"

### Imports

In [ ]:
import json
from math import log
import numpy as np
from os.path import join as __join
import pandas as pd
import time

import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.parsimonious import pfba as pfba_cobra

### Model

In [ ]:
def load_model_add_constraints(cobra_model_file):
    cobra_model = cobra.io.load_json_model(cobra_model_file)
    cobra_model.reactions.EX_photon_abs_.lower_bound = -331
    return(cobra_model)

# Simualtions

### Conditions and experimental growth rates

In [ ]:
cond_1 = "BIOMASS_biomass_WT526_c"

In [ ]:
cond_2 = "BIOMASS_biomass_MgACSBG31_c"

In [ ]:
dict_experimental_growth = {"BIOMASS_biomass_WT526_c": 3.04E-02 ,
                            "BIOMASS_biomass_MgACSBG31_c": 1.50E-02}

# 1) pFBA : Biomass_WT = experimental growth rate, minimize photon uptake

##### run pFBA

In [ ]:
iMgadit23 = load_model_add_constraints(__join(GEM_folder,modelfile))
iMgadit23.objective = "EX_photon_abs_"
for rxn in iMgadit23.reactions:
    if rxn.id[0:16] == "BIOMASS_biomass_":
        if rxn.id == cond_1:
            rxn.lower_bound = dict_experimental_growth[rxn.id]
            rxn.upper_bound = dict_experimental_growth[rxn.id]
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
        else:
            rxn.lower_bound = 0
            rxn.upper_bound = 0
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
start_time = time.time()
sol_pfba_1 = pfba_cobra(iMgadit23)
end_time = time.time()
df_pfba_1 = sol_pfba_1.to_frame()
df_pfba_1.columns = ["Flux pFBA WT","reduced_costs pFBA WT"]

print("--- %s seconds ---" % (end_time - start_time))

In [ ]:
sol_pfba_1

In [ ]:
sol_pfba_1[cond_1]

In [ ]:
sol_pfba_1["EX_photon_abs_"]

##### Save

In [ ]:
# Save fluxes (for Escher)
file = "Res_fluxes_pFBA_WT.json"
with open(file, "w") as f:
    json.dump(sol_pfba_1.fluxes.to_dict(), f)
f.close()

In [ ]:
# csv
df_pfba_1.to_csv("Res_pFBA_WT.csv",sep=";")

In [ ]:
del iMgadit23

# 2) MOMA:Biomass_acsbg = experimental growth rate, minimize diff with pfba_sol1 results

In [ ]:
iMgadit23 = load_model_add_constraints(__join(GEM_folder,modelfile))

### Change EX_ bounds

In [ ]:
ex_to_block = ["EX_arg__L_",
"EX_lys__L_",
"EX_gal_",
"EX_ade_",
"EX_gln__L_",
"EX_for_",
"EX_glc__D_",
"EX_xylt_",
"EX_trp__L_",
"EX_phe__L_",
"EX_glyc3p_",
"EX_fum_",
"EX_val__L_",
"EX_his__L_",
"EX_inost_",
"EX_ac_",
"EX_chol_",
"EX_gua_",
"EX_glu__L_",
"EX_leu__L_",
"EX_sbt__D_",
"EX_tyr__L_",
"EX_pro__L_",
"EX_fru_",
"EX_glyc_",]

In [ ]:
test = []
for rxn in iMgadit23.reactions:
    if rxn.id[0:2] == "EX":
        if df_pfba_1.loc[rxn.id,"Flux pFBA WT"] == 0:
            print(rxn.id)
            test.append(rxn.id)

In [ ]:
set(test) - set(ex_to_block)

In [ ]:
set(ex_to_block) - set(test)

In [ ]:
### set EX_bounds to 0
for rxn_id in ex_to_block:
    rxn = iMgadit23.reactions.get_by_id(rxn_id)
    rxn.lower_bound = 0
    rxn.upper_bound = 0

In [ ]:
iMgadit23.objective = "EX_photon_abs_"
for rxn in iMgadit23.reactions:
    if rxn.id[0:16] == "BIOMASS_biomass_":
        if rxn.id == cond_2:
            rxn.lower_bound = dict_experimental_growth[rxn.id]
            rxn.upper_bound = dict_experimental_growth[rxn.id]
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
        else:
            rxn.lower_bound = 0
            rxn.upper_bound = 0
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)

            
start_time2 = time.time()
res_moma = cobra.flux_analysis.moma(iMgadit23,solution = sol_pfba_1,linear=True)
end_time2 = time.time()

print("--- %s seconds ---" % ((end_time - start_time)+(end_time2 - start_time2)))

In [ ]:
df_moma_cond_2 = res_moma.to_frame()

In [ ]:
res_moma["EX_photon_abs_"]

In [ ]:
res_moma[cond_2]

In [ ]:
df_moma_cond_2.columns = ["Flux MOMA acsbg31","reduced_costs MOMA acsbg31"]

#### Save

In [ ]:
# Save fluxes (for Escher)
file = "Res_fluxes_MOMA_acsbg31.json"
with open(file, "w") as f:
    json.dump(res_moma.fluxes.to_dict(), f)
f.close()

In [ ]:
# csv
df_moma_cond_2.to_csv("Res_MOMA_acsbg31.csv",sep=";")

In [ ]:
del iMgadit23

# 3) pFBA : Biomass_acsbg = experimental growth rate, minimize photon uptake

##### run pFBA

In [ ]:
iMgadit23 = load_model_add_constraints(__join(GEM_folder,modelfile))
iMgadit23.objective = "EX_photon_abs_"
for rxn in iMgadit23.reactions:
    if rxn.id[0:16] == "BIOMASS_biomass_":
        if rxn.id == cond_2:
            rxn.lower_bound = dict_experimental_growth[rxn.id]
            rxn.upper_bound = dict_experimental_growth[rxn.id]
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
        else:
            rxn.lower_bound = 0
            rxn.upper_bound = 0
            print(rxn.id, "bounds = ",rxn.lower_bound,rxn.upper_bound)
            
start_time3 = time.time()
sol_pfba_2 = pfba_cobra(iMgadit23)
end_time3 = time.time()
df_pfba_2 = sol_pfba_2.to_frame()
df_pfba_2.columns = ["Flux pFBA acsbg31","reduced_costs pFBA acsbg31"]

print("--- %s seconds ---" % (end_time3 - start_time3))

In [ ]:
sol_pfba_2

In [ ]:
sol_pfba_2[cond_2]

In [ ]:
sol_pfba_2["EX_photon_abs_"]

#### Save

In [ ]:
# Save fluxes (for Escher)
file = "Res_fluxes_pFBA_acsbg31.json"
with open(file, "w") as f:
    json.dump(sol_pfba_2.fluxes.to_dict(), f)
f.close()

In [ ]:
# csv
df_pfba_2.to_csv("Res_pFBA_acsbg31.csv",sep=";")